### Data Mining - Assignment 1 - Expert Recommendation System

<h3>Team members:</h3>
<center>
    <h4>
        <table>
            <tr>
                <th> Name </th>
                <th> Roll Number </th>
            </tr>
            <tr>
                <td> Kritik Agarwal </td>
                <td> CS23MTECH11009 </td>
            </tr>
            <tr>
                <td> Darpan Gaur </td>
                <td> C021BTECH11004 </td>
            </tr>
            <tr>
                <td> Hari Priyanka Allam </td>
                <td> CC23M24P100001 </td>
            </tr>
            <tr>
                <td> Maloth David </td>
                <td> CS21BTECH11035 </td>
            </tr>
        </table>
    </h4>
</center>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Download the Dataset

In [ ]:
import os
import gdown

# GDrive file link
url = "https://drive.google.com/file/d/19UhOq9Z5IVqM926cC3hvxcl726CTv-kT/view?usp=sharing"

# Output path
output = "/content/drive/MyDrive/softwareengineering.stackexchange.com.7z"

# Check if the file already exists
if not os.path.exists(output):
    print(f"File does not exist. Downloading from {url}...")
    gdown.download(url, output, fuzzy=True)
    print("Download complete.")
else:
    print(f"File already exists at {output}. Skipping download.")


File already exists at /content/drive/MyDrive/softwareengineering.stackexchange.com.7z. Skipping download.


#### Unzip the files and convert to csv

In [ ]:
# Unzip the Dataset File
os.system( '7z x /content/drive/MyDrive/softwareengineering.stackexchange.com.7z -o/content/drive/MyDrive/DataMining/' )

0

In [ ]:
import subprocess

# GitHub repository URL
repo_url = "https://github.com/SkobelevIgor/stackexchange-xml-converter"

# Directory to clone the repository into
clone_dir = "/content/stackexchange-xml-converter"

# Check if the directory already exists
if not os.path.exists(clone_dir):
    print(f"Cloning repository {repo_url} into {clone_dir}...")

    os.system(f"git clone {repo_url} {clone_dir}")
    print("Clone complete.")
else:
    print(f"Repository already exists at {clone_dir}.")

Cloning repository https://github.com/SkobelevIgor/stackexchange-xml-converter into /content/stackexchange-xml-converter...
Clone complete.


In [ ]:
# Function to run a command and handle errors
def run_command(command):
    try:
        subprocess.run(command, check=True)
        return True
    except subprocess.CalledProcessError as e:
        print(f"An error occurred while running {' '.join(command)}: {e}")
        return False

# Install GoLang if not already installed
def install_go():
    print("'go' command not found. Installing Go...")
    return run_command(["apt-get", "install", "golang", "-y"])

# Check if the directory exists
if os.path.exists(clone_dir):
    os.chdir(clone_dir)
    print(f"Changed directory to {clone_dir}")

    try:
        print("Running 'go build'...")
        subprocess.run(["go", "build"], check=True)
        print("Build completed successfully.")
    except FileNotFoundError:
        # If GoLang is not installed, install it
        if install_go():
            print("Go installed successfully. Retrying 'go build'...")
            if run_command(["go", "build"]):
                print("Build completed successfully after Go installation.")
            else:
                print("Build failed after installing Go.")
        else:
            print("Failed to install Go.")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred during the build: {e}")
else:
    print(f"Directory {clone_dir} does not exist. Please clone the repository first.")

Changed directory to /content/stackexchange-xml-converter
Running 'go build'...
'go' command not found. Installing Go...
Go installed successfully. Retrying 'go build'...
Build completed successfully after Go installation.


In [ ]:
import glob

converter_file = "/content/stackexchange-xml-converter/stackexchange-xml-converter"
src_dir = "/content/drive/MyDrive/DataMining/"
dest_dir = "/content/drive/MyDrive/DataMining/"

# Check for XML files in the source directory
xml_files = glob.glob(os.path.join(src_dir, "*.xml"))

if xml_files:
    command = [
        converter_file,
        "-result-format=csv",
        f"-source-path={src_dir}",
        f"-store-to-dir={dest_dir}"
    ]

    try:
        print(f"Running command: {' '.join(command)}")
        subprocess.run(command, check=True)
        print("Conversion completed successfully.")

        # Delete XML files after conversion
        for file in xml_files:
            os.remove(file)
            print(f"Deleted {file}")

    except subprocess.CalledProcessError as e:
        print(f"An error occurred during conversion: {e}")
else:
    print(f"No XML files found in {src_dir}.")

Running command: /content/stackexchange-xml-converter/stackexchange-xml-converter -result-format=csv -source-path=/content/drive/MyDrive/DataMining/ -store-to-dir=/content/drive/MyDrive/DataMining/
Conversion completed successfully.
Deleted /content/drive/MyDrive/DataMining/Badges.xml
Deleted /content/drive/MyDrive/DataMining/Comments.xml
Deleted /content/drive/MyDrive/DataMining/PostHistory.xml
Deleted /content/drive/MyDrive/DataMining/PostLinks.xml
Deleted /content/drive/MyDrive/DataMining/Posts.xml
Deleted /content/drive/MyDrive/DataMining/Tags.xml
Deleted /content/drive/MyDrive/DataMining/Users.xml
Deleted /content/drive/MyDrive/DataMining/Votes.xml


#### Import the Required Libraries

In [ ]:
%pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357269 sha256=6d72af14b045a740083e4d464b491ddd3602148745f7438136d3e15f68e94b2e
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [20]:
import numpy as np
import pandas as pd
import os

from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error

from surprise import KNNBaseline
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise import SVD

#### Load the CSV files

In [ ]:
folder_path = '/content/drive/MyDrive/DataMining/'

csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    df = pd.read_csv(file_path)
    print(f'Loaded {csv_file}')
    print(df.head())

Loaded PostLinks.csv
         Id  RelatedPostId  PostId  LinkTypeId             CreationDate
0   6613628         261243  269546           1  2015-01-09T08:46:25.567
1  10927609         335738  397157           1  2019-09-09T11:46:30.077
2  10927608         377985  397157           1  2019-09-09T11:43:52.163
3  10945537         377985  397157           3  2019-09-19T00:59:43.647
4  11272483         321920  411220           1  2020-06-09T09:08:40.440
Loaded Tags.csv
   Id  ExcerptPostId  WikiPostId                TagName  Count
0   1        84634.0     84633.0               comments    199
1   3       133740.0    133739.0             code-smell    171
2   4         4802.0      4801.0  programming-languages   1403
3   5            NaN         NaN                  usage      8
4   7       109610.0    109609.0               business    127
Loaded Badges.csv
   Id  UserId  Class     Name  TagBased                     Date
0  12      11      3  Teacher     False  2010-09-01T19:40:38.967
1  21

#### Exercise 1

Top 3 Answerers

In [22]:
posts = pd.read_csv('./data/Posts.csv')

answers = posts[posts['PostTypeId'] == 2]
answerer_table = answers.groupby('OwnerUserId').size().reset_index(name='AnswerCount')
answerer_table['OwnerUserId'] = answerer_table['OwnerUserId'].astype(int)
top_3_answerers = answerer_table.sort_values(by='AnswerCount', ascending=False).head(3)

print("Top 3 Answerers:")
print(top_3_answerers)

Top 3 Answerers:
       OwnerUserId  AnswerCount
3189          9113         2839
19912       177980         2326
557           1204         2043


Top 3 Tags

In [23]:
questions = posts[posts['PostTypeId'] == 1]
questions.loc[:, 'Tags'] = questions['Tags'].apply(lambda x: str(x).strip('|').split('|'))

tags_exploded = questions.explode('Tags')
tags_exploded = tags_exploded[tags_exploded['Tags'].notnull() & (tags_exploded['Tags'] != '')]
tags_table = tags_exploded.groupby('Tags').size().reset_index(name='TagCount')
top_3_tags = tags_table.sort_values(by='TagCount', ascending=False).head(3)

print("\nTop 3 Tags:")
print(top_3_tags)


Top 3 Tags:
       Tags  TagCount
415  design      5162
187      c#      4931
773    java      4928


#### Exercise 2

In [24]:
questions = posts[posts['PostTypeId'] == 1][['Id', 'Tags']]

answers = posts[posts['PostTypeId'] == 2][['OwnerUserId', 'ParentId']].dropna(subset=['OwnerUserId'])
answers['OwnerUserId'] = answers['OwnerUserId'].astype(int)

answerer_counts = answers.groupby('OwnerUserId').size().reset_index(name='AnswerCount')
expert_answerers = answerer_counts[answerer_counts['AnswerCount'] >= 20]['OwnerUserId']

questions['Tags'] = questions['Tags'].fillna('').str.strip('|').str.split('|')
exploded_tags = questions.explode('Tags')

tag_counts = exploded_tags.groupby('Tags').size().reset_index(name='TagCount')
popular_tags = tag_counts[tag_counts['TagCount'] >= 20]['Tags']

answerer_tags = pd.merge(answers, exploded_tags, left_on='ParentId', right_on='Id', how='left')

filtered_data = answerer_tags[
    (answerer_tags['OwnerUserId'].isin(expert_answerers)) &
    (answerer_tags['Tags'].isin(popular_tags))
]

utility_matrix = pd.pivot_table(
    filtered_data, index='OwnerUserId', columns='Tags', aggfunc='size', fill_value=0
)

utility_matrix.sort_index(inplace=True)
utility_matrix.sort_index(axis=1, inplace=True)

print("Number of expert answerers (>=20 answers):", len(expert_answerers))
print("Number of popular tags (>=20 annotations):", len(popular_tags))

print("\nDimension of the Expert utility matrix:")
print(utility_matrix.shape)

print("\nUtility Matrix:")
print(utility_matrix)

Number of expert answerers (>=20 answers): 1163
Number of popular tags (>=20 annotations): 974

Dimension of the Expert utility matrix:
(1163, 974)

Utility Matrix:
Tags         .net  .net-core  3d  64-bit  abstract-class  abstract-factory  \
OwnerUserId                                                                  
4               6          0   0       0               0                 1   
6               0          0   0       0               0                 0   
11              1          0   0       0               0                 0   
14              0          0   0       0               0                 0   
15              0          0   0       0               0                 0   
...           ...        ...  ..     ...             ...               ...   
356695          1          0   0       0               0                 0   
366014          0          0   0       0               0                 0   
373864          0          0   0       0               

#### Exercise 3

In [25]:
# Create the utility matrix by modifying each entry of the Expert matrix
utility_matrix = utility_matrix.map(lambda x: x//3 if x < 15 else 5)

In [26]:
# Test data split (bottom 15% of experts and rightmost 15% tags)
test_experts = int(0.15 * utility_matrix.shape[0])
test_tags = int(0.15 * utility_matrix.shape[1])

test_matrix = utility_matrix.iloc[-test_experts:, -test_tags:]
train_matrix = utility_matrix.iloc[:-test_experts, :-test_tags]

In [27]:
# Calculate the required values
utility_sum = utility_matrix.sum().sum()
highest_row_sum = utility_matrix.sum(axis=1).max()
highest_col_sum = utility_matrix.sum(axis=0).max()

train_sum = train_matrix.sum().sum()
test_dim = test_matrix.shape
test_sum = test_matrix.sum().sum()

print("Utility Matrix:")
print(f"Summation value: {utility_sum}")
print(f"Highest row sum: {highest_row_sum}")
print(f"Highest column sum: {highest_col_sum}")

print("\nTrain and Test Data:")
print(f"Train matrix summation value: {train_sum}")
print(f"Test matrix dimension: {test_dim}")
print(f"Test matrix summation value: {test_sum}")

Utility Matrix:
Summation value: 41403
Highest row sum: 1164
Highest column sum: 1403

Train and Test Data:
Train matrix summation value: 28713
Test matrix dimension: (174, 146)
Test matrix summation value: 899


#### Exercise 4

In [ ]:
# Calculate Item-Item Similarity Matrix
def calculate_item_similarity(matrix):
    return matrix.T.corr(method='pearson')

item_similarity_matrix = calculate_item_similarity(utility_matrix)
user_similarity_matrix = utility_matrix.corr(method='pearson')

In [ ]:
# Predict Rating for Item-Item (Simple Average)
def predict_rating_simple_avg(user_id, tag_id, N):
    if tag_id not in item_similarity_matrix.columns:
        return 0
    similar_tags = item_similarity_matrix[tag_id].nlargest(N+1).index[1:N+1]
    ratings = utility_matrix.loc[user_id, similar_tags]
    return ratings.mean() if not ratings.empty else 0

In [ ]:
# Predict Rating for Item-Item (Weighted Average)
def predict_rating_weighted_avg(user_id, tag_id, N):
    if tag_id not in item_similarity_matrix.columns:
        return 0
    similar_tags = item_similarity_matrix[tag_id].nlargest(N+1).index[1:N+1]
    weights = item_similarity_matrix[tag_id][similar_tags]
    ratings = utility_matrix.loc[user_id, similar_tags]
    valid_ratings = ratings.notnull()
    weighted_sum = (ratings[valid_ratings] * weights[valid_ratings]).sum()
    weight_sum = weights[valid_ratings].sum()
    return weighted_sum / weight_sum if weight_sum != 0 else 0

In [ ]:
# Predict Rating for User-User (Simple Average)
def predict_rating_simple_avg_user(user_id, tag_id, N):
    if user_id not in user_similarity_matrix.index: # Check if user_id exists in the index
        return 0
    if tag_id not in utility_matrix.columns:
        return 0
    similar_users = user_similarity_matrix.loc[user_id].nlargest(N+1).index[1:N+1]
    ratings = utility_matrix.loc[similar_users, tag_id]
    return ratings.mean() if not ratings.empty else 0

In [ ]:
# Predict Rating for User-User (Weighted Average)
def predict_rating_weighted_avg_user(user_id, tag_id, N):
    if user_id not in user_similarity_matrix.index: # Check if user_id exists in the index
        return 0
    if tag_id not in utility_matrix.columns:
        return 0
    similar_users = user_similarity_matrix.loc[user_id].nlargest(N+1).index[1:N+1]
    weights = user_similarity_matrix.loc[user_id, similar_users]
    ratings = utility_matrix.loc[similar_users, tag_id]
    valid_ratings = ratings.notnull()
    weighted_sum = (ratings[valid_ratings] * weights[valid_ratings]).sum()
    weight_sum = weights[valid_ratings].sum()
    return weighted_sum / weight_sum if weight_sum != 0 else 0

In [ ]:
# Evaluate RMSE for Item-Item
def evaluate_rmse(predict_function, N):
    actual_ratings = []
    predicted_ratings = []

    for user_id in test_matrix.index:
        for tag_id in test_matrix.columns:
            if not pd.isnull(test_matrix.loc[user_id, tag_id]):
                actual_ratings.append(test_matrix.loc[user_id, tag_id])
                predicted_ratings.append(predict_function(user_id, tag_id, N))

    return np.sqrt(mean_squared_error(actual_ratings, predicted_ratings))

In [ ]:
# Evaluate RMSE for User-User
def evaluate_rmse_user(predict_function, N):
    actual_ratings = []
    predicted_ratings = []
    for user_id in test_matrix.index:
        for tag_id in test_matrix.columns:
            if not pd.isnull(test_matrix.loc[user_id, tag_id]):
                actual_ratings.append(test_matrix.loc[user_id, tag_id])
                predicted_ratings.append(predict_function(user_id, tag_id, N))

    return np.sqrt(mean_squared_error(actual_ratings, predicted_ratings))

In [ ]:
def display_results():
    N_values = [2, 3, 5]

    print("Method | Rating Prediction Function | Metric | N=2 | N=3 | N=5")
    print("------------------------------------------------------------")

    for method, predict_simple, predict_weighted in [
        ("Item-Item", predict_rating_simple_avg, predict_rating_weighted_avg),
        ("User-User", predict_rating_simple_avg_user, predict_rating_weighted_avg_user)
    ]:
        print(f"{method} | Simple Average | RMSE |", end=" ")
        for N in N_values:
            rmse = evaluate_rmse(predict_simple, N) if method == "Item-Item" else evaluate_rmse_user(predict_simple, N)
            print(f"{rmse:.4f} |", end=" ")
        print()

        print(f"{method} | Weighted Average | RMSE |", end=" ")
        for N in N_values:
            rmse = evaluate_rmse(predict_weighted, N) if method == "Item-Item" else evaluate_rmse_user(predict_weighted, N)
            print(f"{rmse:.4f} |", end=" ")
        print()

display_results()

Method | Rating Prediction Function | Metric | N=2 | N=3 | N=5
------------------------------------------------------------
Item-Item | Simple Average | RMSE | 0.3078 | 0.3078 | 0.3078 | 
Item-Item | Weighted Average | RMSE | 0.3078 | 0.3078 | 0.3078 | 
User-User | Simple Average | RMSE | 0.3078 | 0.3078 | 0.3078 | 
User-User | Weighted Average | RMSE | 0.3078 | 0.3078 | 0.3078 | 


#### Exercise 5

In [ ]:
regularization_params = [(0.001, 0.003), (0.05, 0.05), (0.50, 0.75)]
K_values = [2, 5, 10]
num_iterations = 1000
learning_rate = 0.0005
tolerance = 1e-4

In [ ]:
# Define RMSE calculation function
def calculate_rmse(user_item_matrix, P, Q):
    predictions = np.dot(P, Q.T)
    actual = user_item_matrix[user_item_matrix > 0]
    predicted = predictions[user_item_matrix > 0]
    return np.sqrt(mean_squared_error(actual, predicted))

In [ ]:
# Define optimized SGD with and without regularization
def sgd(utility_matrix, P, Q, learning_rate, lambda1=0, lambda2=0):
    num_users, num_items = utility_matrix.shape

    for iteration in range(num_iterations):
        # Vectorized dot product for predictions
        predictions = np.dot(P, Q.T)
        error_matrix = (utility_matrix > 0) * (utility_matrix - predictions)

        # Update P and Q
        P += learning_rate * (np.dot(error_matrix, Q) - lambda1 * P)
        Q += learning_rate * (np.dot(error_matrix.T, P) - lambda2 * Q)

        # Check for convergence (if error is reducing)
        cost = np.linalg.norm((utility_matrix - np.dot(P, Q.T)) * (utility_matrix > 0))
        if cost < tolerance:
            break

    return P, Q

results_df = pd.DataFrame(columns=["Method", "Metric", "K=2", "K=5", "K=10"])

In [ ]:
# Loop through different K values and regularization settings
for regularized, (lambda1, lambda2) in [(False, (0, 0))] + [(True, l1_l2) for l1_l2 in regularization_params]:
    for K in K_values:
        num_users, num_items = utility_matrix.shape
        P = np.random.rand(num_users, K)
        Q = np.random.rand(num_items, K)

        P, Q = sgd(utility_matrix.values, P, Q, learning_rate, lambda1, lambda2)

        rmse = calculate_rmse(utility_matrix.values, P, Q)

        if not regularized:
            results_df.loc[len(results_df)] = ["Without Regularisation", "RMSE", rmse if K == 2 else "", rmse if K == 5 else "", rmse if K == 10 else ""]
        else:
            method = f"With Regularisation λ1={lambda1}, λ2={lambda2}"
            results_df.loc[len(results_df)] = [method, "RMSE", rmse if K == 2 else "", rmse if K == 5 else "", rmse if K == 10 else ""]

print(results_df)

                                    Method Metric       K=2       K=5  \
0                   Without Regularisation   RMSE  0.874779             
1                   Without Regularisation   RMSE            0.734253   
2                   Without Regularisation   RMSE                       
3   With Regularisation λ1=0.001, λ2=0.003   RMSE  0.880756             
4   With Regularisation λ1=0.001, λ2=0.003   RMSE            0.739713   
5   With Regularisation λ1=0.001, λ2=0.003   RMSE                       
6     With Regularisation λ1=0.05, λ2=0.05   RMSE  0.883248             
7     With Regularisation λ1=0.05, λ2=0.05   RMSE            0.742098   
8     With Regularisation λ1=0.05, λ2=0.05   RMSE                       
9      With Regularisation λ1=0.5, λ2=0.75   RMSE  0.878045             
10     With Regularisation λ1=0.5, λ2=0.75   RMSE            0.748843   
11     With Regularisation λ1=0.5, λ2=0.75   RMSE                       

        K=10  
0             
1             
2   0

#### Exercise 6

In [ ]:
sim_options = {
    "name": "pearson_baseline",
    "user_based": False,
}

# Load the data from the utility
reader = Reader(rating_scale=(0, 5))
train_data = Dataset.load_from_df(train_matrix.stack().reset_index(), reader)
test_data = Dataset.load_from_df(test_matrix.stack().reset_index(), reader)
trainset = train_data.build_full_trainset()
testset = test_data.build_full_trainset().build_testset()

# run model for k = 2, 3, 5
for k in [2, 3, 5]:
    model = KNNBaseline(k=k, sim_options=sim_options)
    model.fit(trainset)
    predictions = model.test(testset)
    rmse = accuracy.rmse(predictions)
    print(f"RMSE for k={k}: {rmse}")

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.3058
RMSE for k=2: 0.3057722100059844
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.3058
RMSE for k=3: 0.3057722100059844
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.3058
RMSE for k=5: 0.3057722100059844


In [ ]:
train_data = Dataset.load_from_df(train_matrix.stack().reset_index(), reader)
test_data = Dataset.load_from_df(test_matrix.stack().reset_index(), reader)


# n_epochs = [100, 200, 300, 500]
# lr_all = [0.001, 0.005, 0.01]
# reg_pu = [0.01, 0.02, 0.05]
# reg_qi = [0.01, 0.02, 0.05]

# for n in n_epochs:
#     for lr in lr_all:
#         for pu in reg_pu:
#             for qi in reg_qi:
#                 model = SVD(n_factors=5, lr_all=lr, n_epochs=n, reg_pu=pu, reg_qi=qi)
#                 model.fit(train_data.build_full_trainset())
#                 predictions = model.test(test_data.build_full_trainset().build_testset())
#                 rmse = accuracy.rmse(predictions)
#                 print(f"RMSE for n_epochs={n}, lr_all={lr}, reg_pu={pu}, reg_qi={qi}: {rmse:.4f}")

# run svd model for 3 n_factors :- 2, 5, 10
for n_factors in [2, 5, 10]:
    model = SVD(n_factors=n_factors, lr_all = 0.005, n_epochs=1000)
    model.fit(train_data.build_full_trainset())
    predictions = model.test(test_data.build_full_trainset().build_testset())
    rmse = accuracy.rmse(predictions)
    print(f"RMSE for n_factors={n_factors}: {rmse:.4f}")


RMSE: 0.3058
RMSE for n_factors=2: 0.3058
RMSE: 0.3058
RMSE for n_factors=5: 0.3058
RMSE: 0.3058
RMSE for n_factors=10: 0.3058
